In [1]:
#Install elastic search
!pip install elasticsearch


In [2]:
!pip install -q pandas openai

In [3]:
# To search documents 

#Import Elascticsearch and helpers from  elasticsearch

from elasticsearch import Elasticsearch,helpers
import json

es = Elasticsearch(['http://localhost:9200'])
es.ping()   #connection testing
res = es.search(index="spm587fa24_scm587fa24", body={"query": {"match_all": {}}})

# Extract the total hit count
print("Got %d Hits:" % res['hits']['total']['value'])

# Extract hits from the response
hits = res.get('hits', {})

# Pretty print the hits
print(json.dumps(hits, indent=2))


Got 1303 Hits:
{
  "total": {
    "value": 1303,
    "relation": "eq"
  },
  "max_score": 1.0,
  "hits": [
    {
      "_index": "spm587fa24_scm587fa24",
      "_id": "JFeH3pIBZ0wazt5X3Tfa",
      "_score": 1.0,
      "_source": {
        "_type": "issue",
        "_repo": "SCM587FA24",
        "_issueNumber": "649",
        "_title": "Fixes #639. SFA24SCM52B Bezawada",
        "_createdAt": "2024-10-30T16:50:15Z",
        "_closedAt": "2024-12-31T00:36:30Z",
        "_state": "open",
        "_body": "Phase 2",
        "GitHub_Issue_vector": [
          0.0006351337069645524,
          -0.0040907831862568855,
          -0.00852726586163044,
          0.008894572965800762,
          -0.033936213701963425,
          0.02285941131412983,
          -0.026618899777531624,
          0.0073893372900784016,
          -0.03396502509713173,
          -0.008095141500234604,
          0.0272526815533638,
          -0.008728924207389355,
          0.0128701226785779,
          0.020266661420464516

In [ ]:
# Define model and embedding function
from openai import OpenAI
EMBEDDING_MODEL = "text-embedding-ada-002"
client = OpenAI(api_key="Enter your OPENAPI key")
def get_embedding(text, model="text-embedding-ada-002"):
   text = text.replace("\n", " ")
   return client.embeddings.create(input = [text], model=model).data[0].embedding


In [5]:
# Pretty print function for GitHub Issues

def pretty_response_Issue(response):
    for hit in response['hits']['hits']:
        type = hit['_source']['_type']
        repo =hit['_source']['_repo']
        issueNumber = hit['_source']['_issueNumber']
        title = hit['_source']['_title']
        createdAt = hit['_source']['_createdAt']
        closedAt = hit['_source']['_closedAt']
        state = hit['_source']['_state']
        body = hit['_source']['_body']
        
        pretty_output = (
            f"\nType: {type}\nRepo:{repo}\nIssue Number: {issueNumber}\nTitle: {title}\nCreated At: {createdAt}\nClosed At: {closedAt}\n"
            f"State: {state}\nBody: {body}\n"
        )
        print(pretty_output)

## Lets retrieve React Issues
- Notice React issues are listed first
- After that other Rendering issues are listed

In [6]:
#Searching based on githubissues index 
#This is an example for the semantic search.
# Define question
#Requirement 3
question = 'Find the top 5 most similar issues in the repository'

question_embedding = get_embedding(question, model=EMBEDDING_MODEL)

response = es.search(
  index="spm587fa24_scm587fa24",
  knn={
      "field": "GitHub_Issue_vector",  
      "query_vector": question_embedding,
      "k": 5,
      "num_candidates": 100
    }
)

pretty_response_Issue(response)



Type: issue
Repo:SCM587FA24
Issue Number: 309
Title: OpenAI-cookbook 
Created At: 2024-10-24T03:09:09Z
Closed At: 2024-10-24T04:37:31Z
State: closed
Body: {
  "Title": "Enhance OpenAI-cookbook Documentation for GPT-4 API Integration",
  "Body": "The current OpenAI-cookbook lacks comprehensive examples for integrating the GPT-4 API into various applications. This issue suggests adding detailed use cases, including response parsing, error handling, and optimal prompt construction strategies to better guide developers. The enhanced documentation will help developers efficiently implement the GPT-4 API in different languages and frameworks."
}



Type: issue
Repo:SCM587FA24
Issue Number: 309
Title: OpenAI-cookbook 
Created At: 2024-10-24T03:09:09Z
Closed At: 2024-10-24T04:37:31Z
State: closed
Body: {
  "Title": "Enhance OpenAI-cookbook Documentation for GPT-4 API Integration",
  "Body": "The current OpenAI-cookbook lacks comprehensive examples for integrating the GPT-4 API into various ap

## Lets retrieve Rendering Issues
- Notice Rendering Issues are listed first
- After that other React issues are listed

In [7]:
#Searching based on githubissues index 
#This is an example for the semantic search.
# Define question
#Requirement 4
question = 'Find the top 5 similar issues related to Rendering in the React repository'

question_embedding = get_embedding(question, model=EMBEDDING_MODEL)

response = es.search(
  index="spm587fa24_scm587fa24",
  knn={
      "field": "GitHub_Issue_vector",  
      "query_vector": question_embedding,
      "k": 5,
      "num_candidates": 100
    }
)

pretty_response_Issue(response)



Type: issue
Repo:SCM587FA24
Issue Number: 390
Title: Optimize React Component Re-rendering for Improved Performance
Created At: 2024-10-24T20:41:34Z
Closed At: 2024-10-27T02:18:00Z
State: closed
Body: React's component re-rendering mechanism ensures that UI updates efficiently when state or props change. However, in certain cases, unnecessary re-renders can lead to performance bottlenecks, especially in large applications. This issue proposes enhancing React's internal diffing algorithm and providing better documentation for developers to identify and prevent redundant re-renders. Additionally, introducing more granular tools for performance profiling and visualizing component re-rendering patterns would help developers optimize their applications, leading to faster load times and improved user experience.


Type: issue
Repo:SCM587FA24
Issue Number: 390
Title: Optimize React Component Re-rendering for Improved Performance
Created At: 2024-10-24T20:41:34Z
Closed At: 2024-10-27T02:18:00

In [8]:
# Define the question for Requirement 5
question = 'Find the top 5 Rendering React issues created between 10/23/2024 and 10/25/2024'

# Generate the embedding for the query
question_embedding = get_embedding(question, model=EMBEDDING_MODEL)

# Perform the search with k-NN and date range filter
response = es.search(
  index="spm587fa24_scm587fa24",
  body={
    "size": 5,  # Retrieve the top 5 results
    "query": {
      "bool": {
        "filter": {
          "range": {
            "_createdAt": {
              "gte": "2024-10-23",  # Start date
              "lte": "2024-10-25"   # End date
            }
          }
        }
      }
    },
    "knn": {
      "field": "GitHub_Issue_vector",
      "query_vector": question_embedding,
      "k": 5,
      "num_candidates": 100
    }
  }
)

# Pretty print the top 5 issues
pretty_response_Issue(response)



Type: issue
Repo:SCM587FA24
Issue Number: 227
Title: Bug in Component Rendering
Created At: 2024-10-23T20:46:23Z
Closed At: 2024-10-23T21:02:24Z
State: closed
Body: There is an issue with the component rendering in the latest version of the React framework. The specific issue arises when handling state updates, which leads to unexpected behavior in the UI.


Type: issue
Repo:SCM587FA24
Issue Number: 227
Title: Bug in Component Rendering
Created At: 2024-10-23T20:46:23Z
Closed At: 2024-10-23T21:02:24Z
State: closed
Body: There is an issue with the component rendering in the latest version of the React framework. The specific issue arises when handling state updates, which leads to unexpected behavior in the UI.


Type: issue
Repo:SCM587FA24
Issue Number: 390
Title: Optimize React Component Re-rendering for Improved Performance
Created At: 2024-10-24T20:41:34Z
Closed At: 2024-10-27T02:18:00Z
State: closed
Body: React's component re-rendering mechanism ensures that UI updates efficiently

## Lets retrieve OpenAI-cookbook Issues
- Notice OpenAI-cookbook issues are listed first
- After that other Rate-limit issues are listed

## Sanity Test:

Execute following commands from the command window/terminal:

**To verify the index got created:**
- curl -XGET "http://localhost:9200/_cat/indices?v"

**To check the structure of index:**
- curl -X GET "localhost:9200/github_issues?pretty"

**To check the stats of the index:**

- curl -X GET "localhost:9200/github_issues/_stats?pretty"

**To dump the documents in the index to stdout:**
- curl -XGET "http://localhost:9200/github_issues/_search?pretty=true"

**To dump the mapping of  the index to stdout:**
- curl -XGET "localhost:9200/github_issues/_mapping?pretty"


## How to delete  the index?
**To delete the index you created:**
- curl -XDELETE "http://localhost:9200/github_issues?pretty"

